# Práctica Final: Asistente Turístico de Tenerife (RAG)

Notebook de desarrollo paso a paso.

## 1. Configuración Inicial
Verificación de que el entorno y las claves API están correctamente cargadas.

In [ ]:
import os
import sys
from dotenv import load_dotenv

# 1. Configuramos el path para poder importar módulos de src/
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Cargamos variables de entorno
load_dotenv(os.path.join(project_root, '.env'))

# 3. Verificación
print(f"Directorio Raíz: {project_root}")

api_key = os.getenv("OPENAI_API_KEY")
if api_key and api_key.startswith("sk-"):
    print("✅ API Key de OpenAI cargada correctamente.")
else:
    print("❌ ERROR: No se detectó la API Key o el formato es incorrecto.")

In [ ]:
from langchain_openai import ChatOpenAI

print("⏳ Probando conexión con OpenAI...")
try:
    # Instanciamos el modelo básico
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = llm.invoke("Di 'Conexión exitosa' si me lees.")
    print(f"✅ Respuesta del LLM: {response.content}")
except Exception as e:
    print(f"❌ Error de conexión con OpenAI: {e}")

## 2. Ingesta de Datos (Lectura PDF)
Cargaremos la guía turística desde la carpeta `data/raw` y procesaremos su contenido.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Ruta al archivo PDF (Ahora en data/raw)
pdf_path = os.path.join(project_root, "data", "raw", "TENERIFE.pdf")

print(f"📄 Buscando PDF en: {pdf_path}")

if os.path.exists(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    print(f"✅ PDF Cargado con éxito. Total páginas: {len(docs)}")
    
    # Verificamos contenido de una página al azar (pág 10, índice 9)
    print("\n--- Muestra de contenido (Pág 10) ---")
    print(docs[9].page_content[:500] + "...")
else:
    print("❌ ERROR: El archivo PDF no existe en la ruta especificada.")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Dividimos el texto en fragmentos (chunks) para que entren en el contexto del LLM
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # Caracteres por chunk
    chunk_overlap=200,  # Solapamiento para no perder contexto entre cortes
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(docs)
print(f"✅ Documento dividido en {len(chunks)} fragmentos (chunks).")
print(f"Tamaño promedio del chunk: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} caracteres.")

## 3. Validación del Módulo `src.data.loader`
Ahora verificamos que el código refactorizado en `src/` funciona igual que el código experimental de arriba.

In [ ]:
from src.data.loader import DataLoader

print("🔄 Probando clase DataLoader refactorizada...")
# Inicializamos el loader con la misma ruta
loader = DataLoader(pdf_path)

# Documentos
docs_refactored = loader.load()

# Chunks
chunks_refactored = loader.split(docs_refactored)

print(f"\n✅ Validación completada: {len(chunks_refactored)} chunks generados mediante la clase encapsulada.")

## 4. Motor Vectorial (Embeddings)
Convertiremos los chunks de texto en vectores numéricos y los almacenaremos en ChromaDB.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import shutil

# Ruta donde guardaremos la base de datos vectorial
chroma_path = os.path.join(project_root, "chroma_db")

# Limpiamos la BD anterior si existe para empezar de cero
if os.path.exists(chroma_path):
    print("🧹 Limpiando base de datos vectorial existente...")
    shutil.rmtree(chroma_path)

print("🧠 Generando embeddings... (Esto puede tardar un poco)")

# Inicializamos Embeddings y VectorStore
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma.from_documents(
    documents=chunks_refactored,
    embedding=embeddings,
    persist_directory=chroma_path
)

print(f"✅ Base de datos vectorial creada en: {chroma_path}")

In [ ]:
# Prueba de búsqueda semántica
query = "¿Qué altura tiene el Teide?"
print(f"🔍 Buscando: '{query}'")

results = vector_store.similarity_search(query, k=3)

for i, res in enumerate(results):
    print(f"\n--- Resultado {i+1} ---")
    print(res.page_content[:300] + "...")

## 5. Validación del Módulo `src.data.vector_store`
Verificamos que la clase `VectorStoreManager` gestiona correctamente la creación y búsqueda.

In [ ]:
from src.data.vector_store import VectorStoreManager

print("🔄 Probando VectorStoreManager refactorizada...")

chroma_path_ref = os.path.join(project_root, "chroma_db_refactored")

manager = VectorStoreManager(persist_directory=chroma_path_ref)

# Crear DB
manager.create_vector_store(chunks_refactored, reset=True)

# Buscar
results_ref = manager.search("¿Dónde se encuentra el auditorio Adán Martín?")

for i, res in enumerate(results_ref):
    print(f"\n--- [Refactor] Resultado {i+1} ---")
    print(res.page_content[:300] + "...")

## 6. Lógica RAG (Retrieval Augmented Generation)
Implementamos la cadena completa: Pregunta -> Búsqueda de Contexto -> Prompt con Contexto -> Respuesta LLM.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos el Prompt del Asistente
template = """
Eres un guía turístico experto en Tenerife, amigable y conocedor.
Usa la siguiente información de contexto para responder a la pregunta del turista.
Si no encuentras la respuesta en el contexto, di amablemente que no tienes esa información, no inventes nada.

Contexto:
{context}

Pregunta del Turista: {question}

Respuesta:
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Obtenemos el Retriever (Buscador) de nuestra clase VectorStoreManager
# Usaremos 'manager' que validamos en el paso anterior y que ya tiene los datos cargados
retriever = manager.get_retriever(k=4)

# 3. Definimos la Cadena RAG (Chain)
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm  # Usamos el 'llm' que instanciamos al pricipio (ChatOpenAI)
    | StrOutputParser()
)

print("✅ Cadena RAG construida exitosamente.")

In [ ]:
# Prueba Real
pregunta = "¿Qué planes culturales recomiendas en Santa Cruz?"

print(f"👤 Pregunta: {pregunta}\n")
print("🤖 Generando respuesta...\n")

respuesta = rag_chain.invoke(pregunta)

print("💬 Respuesta del Asistente:")
print(respuesta)

## 7. Validación del Módulo `src.core.rag_engine`
Ahora probamos la implementación modular de la clase `RagEngine`.

In [ ]:
from src.core.rag_engine import RagEngine

print("🔄 Probando RagEngine refactorizada...")

# Instanciamos el motor usando el manager ya existente
rag_engine = RagEngine(vector_store_manager=manager)

pregunta_2 = "¿Dónde puedo ver ballenas en Tenerife?"
print(f"\n👤 Pregunta: {pregunta_2}")

respuesta_2 = rag_engine.get_response(pregunta_2)

print("\n💬 Respuesta del Asistente [desde módulo]:")
print(respuesta_2)

## 8. Agente Conversacional con Memoria
Ahora vamos a permitir que el asistente recuerde la conversación. Usaremos `RunnableWithMessageHistory` de LangChain.

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from operator import itemgetter

# 1. Actualizamos el Prompt para incluir historial
memory_template = """
Eres un guía turístico experto en Tenerife, amigable y conocedor.
Usa la siguiente información de contexto para responder a la pregunta del turista.
Si no encuentras la respuesta en el contexto, di amablemente que no tienes esa información, no inventes nada.

Contexto:
{context}

Historial de Chat:
{chat_history}

Pregunta del Turista: {question}

Respuesta:
"""
memory_prompt = ChatPromptTemplate.from_template(
    memory_template
)

# 2. Creamos nueva chain con 'chat_history'. 
# USAMOS itemgetter("question") PARA QUE EL RETRIEVER SOLO RECIBA LA PREGUNTA
rag_chain_with_history_base = (
    {
        "context": itemgetter("question") | retriever | format_docs, 
        "question": itemgetter("question"), 
        "chat_history": itemgetter("chat_history")
    }
    | memory_prompt
    | llm
    | StrOutputParser()
)

# *Nota: Gestionaremos el historial manualmente para esta demo simple*
print("✅ Prompt actualizado para soportar historial.")

In [ ]:
# Simulamos una conversación manual
chat_history_str = ""

# Turno 1
q1 = "¿Qué tiempo hace en el sur de la isla?"
print(f"👤 Pregunta 1: {q1}")
# Invocamos pasando el historial vacío
r1 = rag_chain_with_history_base.invoke({"question": q1, "chat_history": chat_history_str})
print(f"🤖 Respuesta 1: {r1}\n")

# Actualizamos historial (Simulado)
chat_history_str += f"Humano: {q1}\nAsistente: {r1}\n"

# Turno 2 (Pregunta contextual)
q2 = "¿Y qué playas recomiendas allí? ('allí' se refiere al sur)"
print(f"👤 Pregunta 2 (Contextual): {q2}")
r2 = rag_chain_with_history_base.invoke({"question": q2, "chat_history": chat_history_str})
print(f"🤖 Respuesta 2: {r2}")

## 9. Validación del Módulo `src.core.agent`
Ahora probamos la clase `TouristAgent` que encapsula automáticamente todo el manejo de memoria.

In [ ]:
from src.core.agent import TouristAgent

print("🔄 Probando TouristAgent (Refactorizado)...")

# Instanciamos el Agente
agent = TouristAgent(vector_store_manager=manager)

# Sesión A: Preguntas sobre el Norte
session_id = "sesion_usuario_1"
resp_1 = agent.get_response("¿Qué se puede visitar en Puerto de la Cruz?", session_id=session_id)
print(f"\n🤖 R1: {resp_1}\n")

resp_2 = agent.get_response("¿Y qué tal el clima allí?", session_id=session_id)
print(f"🤖 R2 (Contextual): {resp_2}")